현재 구현: 소송 유형 분류만 (5개 카테고리)
- 헌법, 형사소송, 행정소송, 민사/가사소송, 기타

여기에 추가 기능:
1. 승소율 예측
2. 형량/벌금 예측 ㄴ
3. 위험도 분석
4. 관련 판례 추출

In [1]:
import pandas as pd
import joblib
import os
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import Ridge

print("필요한 패키지 임포트 완료")

필요한 패키지 임포트 완료


In [2]:
# 데이터 로드
try:
    train_df = pd.read_pickle("../pkl_file/pkl/train_filtered.pkl")
    test_df = pd.read_pickle("../pkl_file/pkl/test_filtered.pkl")
    
    print("✅ 전처리된 데이터를 성공적으로 불러왔습니다.")
    print(f"   학습 데이터: {len(train_df):,}건")
    print(f"   테스트 데이터: {len(test_df):,}건")
    
    # ⭐ 추가: 기타와 헌법 제외
    print("\n🔄 기타와 헌법 제외")
    train_df = train_df[~train_df['case_type'].isin(['기타', '헌법'])].copy()
    test_df = test_df[~test_df['case_type'].isin(['기타', '헌법'])].copy()
    
    print(f"\n✅ 필터링 완료!")
    print(f"   학습 데이터: {len(train_df):,}건")
    print(f"   테스트 데이터: {len(test_df):,}건")
    print("\n[필터링 후 소송 유형 분포]")
    print(train_df['case_type'].value_counts())
    
except FileNotFoundError:
    print("❌ 파일을 찾을 수 없습니다.")
    print("   machine_legal.ipynb에서 데이터를 먼저 저장해주세요.")
    raise

✅ 전처리된 데이터를 성공적으로 불러왔습니다.
   학습 데이터: 48,308건
   테스트 데이터: 5,435건

🔄 기타와 헌법 제외

✅ 필터링 완료!
   학습 데이터: 48,308건
   테스트 데이터: 5,435건

[필터링 후 소송 유형 분포]
case_type
형사소송       37831
행정소송        9433
민사/가사소송     1044
Name: count, dtype: int64


In [3]:
# 셀3. 특성 추출 함수 정의
def classify_outcome(text):
    """판결문에서 승소/패소 분류"""
    win_keywords = ['인용', '승소', '받아들인다', '타당하다', '인정된다']
    lose_keywords = ['기각', '패소', '받아들이지', #'부당하다', 
                     '인정되지']
    
    win_count = sum(1 for k in win_keywords if k in text)
    lose_count = sum(1 for k in lose_keywords if k in text)
    
    if win_count > lose_count:
        return 'win'
    elif lose_count > win_count:
        return 'lose'
    else:
        return 'unknown'

def extract_sentence_years(text):
    """형량 추출 (예: "징역 3년" → 3)"""
    pattern = r'징역\s*(\d+)년'
    match = re.search(pattern, text)
    return int(match.group(1)) if match else 0

def extract_fine(text):
    """벌금 추출"""
    patterns = [
        (r'벌금\s*(\d+)억', 100000000),
        (r'벌금\s*(\d+)천만', 10000000),
        (r'벌금\s*(\d+)백만', 1000000),
        (r'벌금\s*(\d+)만', 10000)
    ]
    
    for pattern, multiplier in patterns:
        match = re.search(pattern, text)
        if match:
            return int(match.group(1)) * multiplier
    return 0

def calculate_risk_score(text):
    """위험도 점수 계산 (0~100)"""
    high_risk_keywords = ['징역', '실형', '구속', '무기', '사형']
    medium_risk_keywords = ['벌금', '과태료', '손해배상', '처분']
    
    high_count = sum(2 for k in high_risk_keywords if k in text)
    medium_count = sum(1 for k in medium_risk_keywords if k in text)
    
    score = min((high_count + medium_count) * 10, 100)
    return score

print("✅ 특성 추출 함수 4개 정의 완료")

✅ 특성 추출 함수 4개 정의 완료


In [4]:
# 셀4. 데이터에 새 특성 추가
print("\n🔄 학습 데이터에 추가 특성 생성 중...")

# 1) 승소/패소 레이블
train_df['outcome'] = train_df['body'].apply(classify_outcome)
test_df['outcome'] = test_df['body'].apply(classify_outcome)
print("  ✓ 승소/패소 레이블 생성")

# 2) 형량 정보
train_df['sentence_years'] = train_df['body'].apply(extract_sentence_years)
test_df['sentence_years'] = test_df['body'].apply(extract_sentence_years)
print("  ✓ 형량 정보 추출")

# 3) 벌금 정보
train_df['fine_amount'] = train_df['body'].apply(extract_fine)
test_df['fine_amount'] = test_df['body'].apply(extract_fine)
print("  ✓ 벌금 정보 추출")

# 4) 위험도 점수
train_df['risk_score'] = train_df['text'].apply(calculate_risk_score)
test_df['risk_score'] = test_df['text'].apply(calculate_risk_score)
print("  ✓ 위험도 점수 계산")

# 5) 승소율 계산 (outcome 기반)
outcome_map = {'win': 80, 'lose': 20, 'unknown': 50}
train_df['win_rate'] = train_df['outcome'].map(outcome_map)
test_df['win_rate'] = test_df['outcome'].map(outcome_map)
print("  ✓ 승소율 레이블 생성")

print("\n✅ 모든 특성 생성 완료!")
print("\n[생성된 특성 샘플]")
display(train_df[['outcome', 'sentence_years', 'fine_amount', 'risk_score', 'win_rate']].head())

# 통계 확인
print("\n[특성 통계]")
print(f"승소 사건: {(train_df['outcome']=='win').sum():,}건")
print(f"패소 사건: {(train_df['outcome']=='lose').sum():,}건")
print(f"불명확: {(train_df['outcome']=='unknown').sum():,}건")
print(f"형량 있는 사건: {(train_df['sentence_years']>0).sum():,}건")
print(f"벌금 있는 사건: {(train_df['fine_amount']>0).sum():,}건")


🔄 학습 데이터에 추가 특성 생성 중...
  ✓ 승소/패소 레이블 생성
  ✓ 형량 정보 추출
  ✓ 벌금 정보 추출
  ✓ 위험도 점수 계산
  ✓ 승소율 레이블 생성

✅ 모든 특성 생성 완료!

[생성된 특성 샘플]


,outcome,sentence_years,fine_amount,risk_score,win_rate
78,unknown,0,0,10,50
79,lose,0,0,10,20
80,lose,0,0,10,20
81,lose,0,0,10,20
83,unknown,0,0,30,50



[특성 통계]
승소 사건: 465건
패소 사건: 7,101건
불명확: 40,742건
형량 있는 사건: 10,747건
벌금 있는 사건: 4,252건


In [5]:
# 셀5. LegalAIPredictor 클래스 정의
class LegalAIPredictor:
    """법률 AI 종합 예측 모델"""
    
    def __init__(self):
        self.case_type_model = None
        self.win_rate_model = None
        self.sentence_model = None
        self.risk_model = None
        self.fine_model = None
        self.vectorizer = None
        
    # def fit(self, X_text, y_case_type, y_win_rate, y_sentence, y_risk, y_fine):
    def fit(self, X_text, y_win_rate, y_sentence, y_risk, y_fine): #소송분류 제거
        """모든 모델 학습"""
        print("\n🔄 모델 학습 시작...")
        
        # 텍스트 벡터화 (단어 개수를 10,000개로 조정하여 속도 극대화)
        self.vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), min_df=5)
        X_vec = self.vectorizer.fit_transform(X_text)
        print(f"  → 벡터화 완료! 크기: {X_vec.shape}")
        
        # # 1) 소송 유형 분류
        # print("  1/5 소송 유형 분류 모델 학습 중...")
        # self.case_type_model = LogisticRegression(max_iter=1000, class_weight='balanced')
        # self.case_type_model.fit(X_vec, y_case_type)
        
        # 2) 승소율 예측 랜덤포레스트-> 릿지로 변경/텍스트에 강하고 빠름.
        # print("  2/5 승소율 예측 모델 학습 중...")
        # self.win_rate_model = RandomForestRegressor(n_estimators=100, random_state=42)
        # self.win_rate_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        # self.win_rate_model.fit(X_vec, y_win_rate)
        print("  → 2/5 승소율 모델 학습 중 (Ridge)...")
        self.win_rate_model = Ridge(alpha=1.0).fit(X_vec, y_win_rate)
        
        
        # 3) 형량 예측
        print("  3/5 형량 예측 모델 학습 중...")
        # self.sentence_model = RandomForestRegressor(n_estimators=100, random_state=42)
        # self.sentence_model.fit(X_vec, y_sentence)
        self.sentence_model = Ridge(alpha=1.0).fit(X_vec, y_sentence)
        
        # 4) 위험도 예측
        print("  4/5 위험도 예측 모델 학습 중...")
        # self.risk_model = RandomForestRegressor(n_estimators=100, random_state=42)
        # self.risk_model.fit(X_vec, y_risk)
        self.risk_model = Ridge(alpha=1.0).fit(X_vec, y_risk)
        
        # 5) 벌금 예측
        print("  5/5 벌금 예측 모델 학습 중...")
        # self.fine_model = RandomForestRegressor(n_estimators=100, random_state=42)
        # self.fine_model.fit(X_vec, y_fine)
        self.fine_model = Ridge(alpha=1.0).fit(X_vec, y_fine)
        
        print("✅ 모든 모델 학습 완료!")
        return self
    
    def predict_all(self, text):
        """모든 예측 수행"""
        X_vec = self.vectorizer.transform([text])
        
        results = {
            # 'case_type': self.case_type_model.predict(X_vec)[0], #소송분류
            # 'case_type_proba': self.case_type_model.predict_proba(X_vec).max(), #무슨소송인지 확률
            'win_rate': min(max(self.win_rate_model.predict(X_vec)[0], 0), 100),
            'sentence_years': max(self.sentence_model.predict(X_vec)[0], 0),
            'fine_amount': max(self.fine_model.predict(X_vec)[0], 0),
            'risk_score': min(max(self.risk_model.predict(X_vec)[0], 0), 100)
        }
        
        return results

print("✅ LegalAIPredictor 클래스 정의 완료")

✅ LegalAIPredictor 클래스 정의 완료


In [6]:
# 셀6. 모델학습
# 학습데이터 준비
X_train = train_df['text'] #사건내용
# y_case_type = train_df['case_type'] #소송분류
y_win_rate = train_df['win_rate'] #승소율
y_sentence = train_df['sentence_years'] #형량예측
y_risk = train_df['risk_score'] #위험도
y_fine = train_df['fine_amount'] #벌금

# 모델 초기화 및 학습
predictor = LegalAIPredictor()
# predictor.fit(X_train, #y_case_type,
#               y_win_rate, y_sentence, y_risk, y_fine)
predictor.fit(X_train, y_win_rate, y_sentence, y_risk, y_fine)

# %% [셀 7: 성능 평가]
print("\n" + "="*60)
print("📊 테스트 데이터 예측 결과")
print("="*60)

# 테스트 데이터로 예측 (전체 데이터)
test_predictions = []
print(f"\n전체 {len(test_df):,}건 예측 중...")

for idx, text in enumerate(test_df['text'], 1):
    if idx % 1000 == 0:
        print(f"  진행: {idx:,}/{len(test_df):,}")
    pred = predictor.predict_all(text)
    test_predictions.append(pred)

# 결과를 DataFrame으로 변환
pred_df = pd.DataFrame(test_predictions)

print("\n[예측 결과 통계]")
print(f"평균 승소율: {pred_df['win_rate'].mean():.1f}%")
print(f"평균 형량: {pred_df['sentence_years'].mean():.2f}년")
print(f"평균 벌금: {pred_df['fine_amount'].mean():,.0f}원")
print(f"평균 위험도: {pred_df['risk_score'].mean():.1f}/100")

# 실제값과 비교
print("\n[실제값 통계]")
print(f"실제 평균 승소율: {test_df['win_rate'].mean():.1f}%")
print(f"실제 평균 형량: {test_df['sentence_years'].mean():.2f}년")
print(f"실제 평균 벌금: {test_df['fine_amount'].mean():,.0f}원")
print(f"실제 평균 위험도: {test_df['risk_score'].mean():.1f}/100")


🔄 모델 학습 시작...


  → 벡터화 완료! 크기: (48308, 10000)
  → 2/5 승소율 모델 학습 중 (Ridge)...
  3/5 형량 예측 모델 학습 중...
  4/5 위험도 예측 모델 학습 중...
  5/5 벌금 예측 모델 학습 중...
✅ 모든 모델 학습 완료!

📊 테스트 데이터 예측 결과

전체 5,435건 예측 중...
  진행: 1,000/5,435
  진행: 2,000/5,435
  진행: 3,000/5,435
  진행: 4,000/5,435
  진행: 5,000/5,435

[예측 결과 통계]
평균 승소율: 45.4%
평균 형량: 0.48년
평균 벌금: 33,394,420원
평균 위험도: 12.9/100

[실제값 통계]
실제 평균 승소율: 45.4%
실제 평균 형량: 0.37년
실제 평균 벌금: 513,358원
실제 평균 위험도: 12.6/100


In [7]:
# 셀8. analyze_case 함수

def analyze_case(user_input):
    """사용자 입력 분석"""
    result = predictor.predict_all(user_input)
    
    print("\n" + "="*60)
    print("🤖 AI 법률 상담 결과")
    print("="*60)
    # print(f"\n📁 소송 유형: {result['case_type']} (확신도: {result['case_type_proba']:.1%})")
    print(f"📊 예상 승소율: {result['win_rate']:.1f}%")
    
    if result['sentence_years'] > 0:
        print(f"⚖️ 예상 형량: {result['sentence_years']:.1f}년")
    
    if result['fine_amount'] > 0:
        print(f"💰 예상 벌금: {result['fine_amount']:,.0f}원")
    
    print(f"⚠️ 위험도: {result['risk_score']:.1f}/100")
    
    # 위험도에 따른 조언
    if result['risk_score'] >= 70:
        print("\n⛔ 높은 위험: 전문 변호사 상담을 강력히 권장합니다.")
    elif result['risk_score'] >= 40:
        print("\n⚠️ 중간 위험: 법률 전문가와 상담하시는 것이 좋습니다.")
    else:
        print("\n✅ 낮은 위험: 기본적인 법률 절차로 대응 가능할 것으로 보입니다.")
    
    print("="*60)
    
    return result

# 테스트 케이스들
print("\n🔍 테스트 케이스 1: 형사 사건")
analyze_case("상대방이 나를 밀어서 나는 전치 4주의 진단을 받았다.")

print("\n🔍 테스트 케이스 2: 민사 사건")
analyze_case("임대인이 보증금 1000만원을 저에게 돌려주지 않아 임차권 등기명령을 신청합니다.")

print("\n🔍 테스트 케이스 3: 행정 사건")
analyze_case("행정청의 영업정지 6개월 처분이 부당하므로 이를 취소해 주시기 바랍니다.")


🔍 테스트 케이스 1: 형사 사건

🤖 AI 법률 상담 결과
📊 예상 승소율: 47.7%
⚠️ 위험도: 6.8/100

✅ 낮은 위험: 기본적인 법률 절차로 대응 가능할 것으로 보입니다.

🔍 테스트 케이스 2: 민사 사건

🤖 AI 법률 상담 결과
📊 예상 승소율: 43.9%
⚠️ 위험도: 5.0/100

✅ 낮은 위험: 기본적인 법률 절차로 대응 가능할 것으로 보입니다.

🔍 테스트 케이스 3: 행정 사건

🤖 AI 법률 상담 결과
📊 예상 승소율: 49.8%
⚠️ 위험도: 15.0/100

✅ 낮은 위험: 기본적인 법률 절차로 대응 가능할 것으로 보입니다.


{'win_rate': np.float64(49.84724613361678),
 'sentence_years': 0,
 'fine_amount': 0,
 'risk_score': np.float64(14.970771008803984)}

In [8]:
# # 셀9. CaseFinder 클래스

# class CaseFinder:
#     """유사 판례 검색"""
    
#     def __init__(self, vectorizer, case_db):
#         self.vectorizer = vectorizer
#         self.case_db = case_db
#         print("  → 판례 벡터화 중...")
#         self.case_vectors = vectorizer.transform(case_db['text'])
#         print(f"    총 {len(case_db):,}개 판례 준비 완료")
        
#     def find_similar_cases(self, query_text, top_k=5):
#         """유사한 판례 찾기"""
#         query_vec = self.vectorizer.transform([query_text])
#         similarities = cosine_similarity(query_vec, self.case_vectors)[0]
        
#         top_indices = similarities.argsort()[-top_k:][::-1]
        
#         results = []
#         for idx in top_indices:
#             results.append({
#                 'case_name': self.case_db.iloc[idx]['case_name'],
#                 'case_num': self.case_db.iloc[idx]['case_num'],
#                 'similarity': similarities[idx],
#                 'summary': self.case_db.iloc[idx]['body'][:200] + "..."
#             })
        
#         return results

# # 판례 검색기 초기화
# print("\n🔍 판례 검색 시스템 초기화 중...")
# case_finder = CaseFinder(predictor.vectorizer, train_df)
# print("✅ 판례 검색 시스템 준비 완료")

# # # 테스트
# # print("\n" + "="*60)
# # print("🔍 유사 판례 검색 테스트")
# # print("="*60)
# # user_query = "임대인이 보증금을 돌려주지 않습니다."
# # similar_cases = case_finder.find_similar_cases(user_query, top_k=3)

# # print(f"\n📝 입력: {user_query}")
# # print("\n[유사 판례 Top 3]")
# # for i, case in enumerate(similar_cases, 1):
# #     print(f"\n{i}. {case['case_name']}")
# #     print(f"   사건번호: {case['case_num']}")
# #     print(f"   유사도: {case['similarity']:.2%}")
# #     print(f"   요약: {case['summary']}")

In [9]:
# 셀10. 모델 저장

# 전체 모델 저장
joblib.dump(predictor, '../pkl_file/machine_data/model/legal_ai_machinModel.pkl')
print("✅ 모델 저장 완료: ../pkl_file/machine_data/model/legal_ai_machinModel.pkl")

# # 판례 검색기도 저장
# joblib.dump(case_finder, '../pkl_file/machine_data/case_finderMachinModel.pkl')
# print("✅ 판례 검색기 저장 완료: ../pkl_file/machine_data/case_finderMachinModel.pkl")

# 업데이트된 데이터프레임도 저장
train_df.to_pickle("../pkl_file/machine_data/train_machineData.pkl")
test_df.to_pickle("../pkl_file/machine_data/test_machineData.pkl")
print("✅ 강화된 데이터 저장 완료")

✅ 모델 저장 완료: ../pkl_file/machine_data/model/legal_ai_machinModel.pkl
✅ 강화된 데이터 저장 완료


In [10]:
# 셀 11. 최종 통합 함수

def full_legal_analysis(user_input):
    # """완전한 법률 분석 (예측 + 판례 검색)"""
    """완전한 법률 분석 (예측)"""
    print("\n" + "="*70)
    print("🏛️ AI 법률 종합 분석 시스템")
    print("="*70)
    
    print(f"\n📝 입력 내용:\n{user_input}\n")
    
    # 1. 예측
    result = predictor.predict_all(user_input)
    
    print("="*70)
    print("📊 분석 결과")
    print("="*70)
    # print(f"\n📁 소송 유형: {result['case_type']} (확신도: {result['case_type_proba']:.1%})")
    print(f"📊 예상 승소율: {result['win_rate']:.1f}%")
    
    if result['sentence_years'] > 0:
        print(f"⚖️ 예상 형량: {result['sentence_years']:.1f}년")
    
    if result['fine_amount'] > 0:
        print(f"💰 예상 벌금: {result['fine_amount']:,.0f}원")
    
    print(f"⚠️ 위험도: {result['risk_score']:.1f}/100")
    
    # # 2. 유사 판례 검색
    # similar_cases = case_finder.find_similar_cases(user_input, top_k=3)
    
    # print("\n" + "="*70)
    # print("📚 관련 판례")
    # print("="*70)
    # for i, case in enumerate(similar_cases, 1):
    #     print(f"\n{i}. {case['case_name']}")
    #     print(f"   사건번호: {case['case_num']}")
    #     print(f"   유사도: {case['similarity']:.2%}")
    
    # 3. 위험도 분석 및 조언
    print("\n" + "="*70)
    print("AI 법률서비스의 의견 *참고용으로 부탁드립니다.")
    print("="*70)
    
    if result['risk_score'] >= 70:
        print("\n⛔ 위험도 : 높음")
        print("  - 즉시 전문 변호사 상담을 받으시기 바랍니다.")
        print("  - 증거 자료를 철저히 준비하세요.")
    elif result['risk_score'] >= 40:
        print("\n⚠️ 위험도 : 중간")
        print("  - 법률 전문가와 상담하시는 것이 좋습니다.")
        print("  - 관련 서류를 미리 준비하세요.")
    else:
        print("\n✅ 위험도 : 비교적 낮음")
        print("  - 기본적인 법률 절차로 대응 가능합니다.")
        print("  - 관련 법률을 숙지하시면 도움이 됩니다.")    
    print("\n" + "="*70 + "\n")
    
    return result #, similar_cases



# 최종 테스트
test_input = """
저는 회사에서 부당해고를 당했습니다.
정당한 사유 없이 갑자기 해고 통보를 받았고,
퇴직금 500만원도 제대로 받지 못했습니다.
근무 기간은 3년입니다.
"""

full_legal_analysis(test_input)


🏛️ AI 법률 종합 분석 시스템

📝 입력 내용:

저는 회사에서 부당해고를 당했습니다.
정당한 사유 없이 갑자기 해고 통보를 받았고,
퇴직금 500만원도 제대로 받지 못했습니다.
근무 기간은 3년입니다.


📊 분석 결과
📊 예상 승소율: 48.1%
💰 예상 벌금: 72,707,900원
⚠️ 위험도: 2.0/100

AI 법률서비스의 의견 *참고용으로 부탁드립니다.

✅ 위험도 : 비교적 낮음
  - 기본적인 법률 절차로 대응 가능합니다.
  - 관련 법률을 숙지하시면 도움이 됩니다.




{'win_rate': np.float64(48.08365126384487),
 'sentence_years': 0,
 'fine_amount': np.float64(72707900.18688074),
 'risk_score': np.float64(1.9798581252768666)}

In [11]:
# 셀 12 완료 메시지

print("\n" + "="*70)
print("🎉 머신러닝 파트 업그레이드 완료!")
print("="*70)
print("\n📌 구현된 기능:")
# print("  ✅ 1. 소송 유형 분류")
print("  ✅  승소율 예측")
print("  ✅  형량 예측")
print("  ✅  벌금 예측")
print("  ✅  위험도 분석")
# print("  ✅  유사 판례 검색")
print("\n📦 저장된 파일:")

print("  - ../pkl_file/machine_data/model/legal_ai_machinModel.pkl")
# print("  - ../pkl_file/machine_data/case_finderMachinModel.pk") #판례 검색기
print("  - ../pkl_file/machine_data/train_machineData.pkl")
print("  - ../pkl_file/machine_data/test_machineData.pkl")
print("\n🔜 다음 단계: 딥러닝 모델 개발 (BERT 활용)")
print("="*70)


🎉 머신러닝 파트 업그레이드 완료!

📌 구현된 기능:
  ✅  승소율 예측
  ✅  형량 예측
  ✅  벌금 예측
  ✅  위험도 분석

📦 저장된 파일:
  - ../pkl_file/machine_data/model/legal_ai_machinModel.pkl
  - ../pkl_file/machine_data/train_machineData.pkl
  - ../pkl_file/machine_data/test_machineData.pkl

🔜 다음 단계: 딥러닝 모델 개발 (BERT 활용)
